# Story Generator

# 1. Ready

## 1.1 기본 패키지 임포트

In [1]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import GPT2LMHeadModel

## 1.2 학습 모델 다운로드

In [2]:
MODEL_NAME = "skt/kogpt2-base-v2"

## 1.3 토크나이저 load

In [3]:
tokenizer = PreTrainedTokenizerFast.from_pretrained(MODEL_NAME)

TOKENS_DICT = {
    'bos_token':'<s>',
    'eos_token':'</s>',
    'unk_token':'<unk>',
    'pad_token':'<pad>',
    'mask_token':'<mask>'
}

# 특수 토큰이 토크나이저에 추가되고 모델은 수정된 토크나이저에 맞게 임베딩의 크기를 조정
tokenizer.add_special_tokens(TOKENS_DICT)

print(tokenizer.special_tokens_map)

{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask>'}


## 1.4 미세조정학습 모델 load

In [5]:
from util.generator import sample_sequence as gen
from util.model import model_loading as load

In [7]:
checkpointPath = "./modelCheckpoint/talestart1890w.tar"
loading = True

In [8]:
model,ckp = load(checkpointPath, PU = 'cpu', status = loading)

In [9]:
print(ckp['epoch'], ckp['loss'])

2000 tensor(0.0307, requires_grad=True)


In [14]:
vocab_path = "./data/정제/index_tale_plus_novel.txt"

In [15]:
p1, p2, p3 = 1,2,1

max_sent = [p1,p2//2, p2//2 ,p3]

context = "어제"


In [16]:
from util.generator import sample_sequence
from util.similarity import similarity

In [17]:

# 1번 문단 생성
print("1번째 문단")
result = []
p = sample_sequence(
    vocab=vocab_path,
    model=model, 
    length = max_sent[0]*30,
    context=context,
    num_samples=1, 
    repetition_penalty=2.0,
    top_p=0.9,
    tokenizer = tokenizer)
p = tokenizer.decode(p[0,:].tolist())
result.append(p)

# 2~4번 문단 생성
for i in range(len(max_sent[1:])):
    print(f"{i+2}번째 문단")
    sim = 0
    while(sim < 0.02):
        next_p = sample_sequence(
            vocab=vocab_path,
            model=model, 
            length=max_sent[i+1]*30,
            context=p, 
            num_samples=1, 
            repetition_penalty=2.0,
            top_p=0.9,
            tokenizer = tokenizer)
        next_p = tokenizer.decode(next_p[0,:].tolist())
        sim = similarity(p, next_p)
        print(sim)

    result.append(next_p)
    p = next_p

1번째 문단


31it [00:03,  8.31it/s]


Word not in corpus => 0 skipped
2번째 문단


31it [00:07,  4.41it/s]


Word not in corpus => 0 skipped
0.5680726713888613
3번째 문단


31it [00:12,  2.51it/s]


Word not in corpus => 4 skipped
0.7499191438281658
4번째 문단


31it [00:13,  2.25it/s]

Word not in corpus => 0 skipped
0.8010912403995811
